In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from libary import *
from recurrent import *
from ARMA import *

In [2]:
train_ratio = 0.5
valid_ratio = 0.25

# Select frequency
frequencies = ['min','5min','15min','60min','D']
frequencies_number_of_samples = np.multiply([390,78,26,7,1],252)

In [3]:
for frequency_index in range(4, -1, -1):
    print(f'Frequency: {frequencies[frequency_index]}')
    number_of_study_periods, study_periods, Data, dates = creating_study_periods(frequencies,\
                                                                                 frequencies_number_of_samples,\
                                                                                 frequency_index)
    # visualize_data(Data, Returns)
    
#     print('ARMA')
#     ARMA_parameters, ARMA_mse, ARMA_predictions = train_ARMA(number_of_study_periods, study_periods,\
#                                                              train_ratio, valid_ratio,\
#                                                              frequency_index, frequencies, frequencies_number_of_samples)
    
#     print('LSTM')
#     LSTM_names, LSTM_mse, LSTM_predictions = train_recurrent_model('LSTM', number_of_study_periods ,study_periods,\
#                                                              train_ratio, valid_ratio,\
#                                                              frequency_index, frequencies, frequencies_number_of_samples)
    print('GRU')
    GRU_names, GRU_mse, GRU_predictions = train_recurrent_model('GRU', number_of_study_periods, study_periods,\
                                                             train_ratio, valid_ratio,\
                                                             frequency_index, frequencies, frequencies_number_of_samples)
    
    
    
    visualize_results((np.concatenate((np.reshape(ARMA_mse[:,-1], [number_of_study_periods,1]),\
                          np.reshape(LSTM_mse[:,-1], [number_of_study_periods,1]),\
                          np.reshape(GRU_mse[:,-1], [number_of_study_periods,1]),\
                          np.reshape(np.mean(np.square(study_periods[0,:,-ARMA_predictions.shape[1]:]),axis=1),\
                                                         [number_of_study_periods,1])), axis=1)))

Frequency: D
GRU
Period: 0
|   iter    |  target   | batch_... |  dropout  | first_... | layer_... | look_back | n_layers  | optimizer |
-------------------------------------------------------------------------------------------------------------
|  1        | -0.000326 |  6.512    |  0.04052  |  38.39    |  0.882    |  38.24    |  2.263    |  1.651    |
|  2        | -0.000330 |  6.937    |  0.2409   |  2.191    |  0.8152   |  28.04    |  2.226    |  0.6594   |
Period time: 1.08 minutes
Period: 1
|   iter    |  target   | batch_... |  dropout  | first_... | layer_... | look_back | n_layers  | optimizer |
-------------------------------------------------------------------------------------------------------------
|  1        | -0.000129 |  5.559    |  0.2283   |  4.048    |  0.636    |  9.824    |  3.377    |  0.5254   |
|  2        | -0.000128 |  9.015    |  0.2145   |  26.5     |  0.7836   |  36.21    |  2.291    |  1.537    |
Period time: 0.97 minutes
Period: 2
|   iter    |  target

NameError: name 'ARMA_mse' is not defined

In [4]:
model_names = ['ARMA', 'LSTM', 'GRU']

for frequency_index in range(4, -1, -1):
    print(f'Frequency: {frequencies[frequency_index]}')
    number_of_study_periods, study_periods, Data, dates = creating_study_periods(frequencies,\
                                                                                 frequencies_number_of_samples,\
                                                                                 frequency_index)
    study_periods_direction = study_periods[0]
    study_periods_direction[0<study_periods_direction] = 1
    study_periods_direction[0==study_periods_direction] = 0
    study_periods_direction[0>study_periods_direction] = -1
    
    train_size = np.round(np.size(study_periods[0], 1) * train_ratio).astype(int)
    valid_size = np.round(np.size(study_periods[0], 1) * valid_ratio).astype(int)
    test_size = (np.size(study_periods[0], 1) - train_size - valid_size).astype(int)
    
    predictions = np.zeros((3,number_of_study_periods,test_size))
    for model_index in range(3):
        print(model_names[model_index])
        predictions[model_index] = pd.read_csv('results/'+str(model_names[model_index])+'_predictions_frequency_'\
                                         +str(frequencies[frequency_index])+'.csv', header=None).dropna(axis='columns').values
print(predictions)

Frequency: D
ARMA
LSTM
GRU
Frequency: 60min
ARMA


ValueError: could not broadcast input array from shape (8,3528) into shape (8,882)

In [ ]:
import statsmodels.api as sm
i = 7
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(study_periods[0,i], lags=40, ax=ax1) # 
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(study_periods[0,i], lags=40, ax=ax2)# , lags=40